# Plan

- preprocess
    - 하나의 파일로 만들기
        - 데이터의 id는 (위도, 경도)
        - 각각 reserve와 store_info 합치기
        - 하나의 csv 파일로 합치기 
        - 요일, holiday flag column 추가
        - 식당 genre, area column 추가 
        - air_visit_data 이용해서 워크인 count

    - feature 찾기
        - one-hot encoding (요일, 식당 genre, area)
        - 분포 확인 (로그 변환 사용)
        - 상관도 검색

- train
    - train : val = 8 : 2
    - 여러 모델 테스트 (앙상블 사용 예상)
    - 하이퍼 파라미터 조정
    - RMSLE 값 기준

- test
    - 케글측에서 일부러 연휴 기간을 포함, 식당이 열지 않은 날도 있지만 최종 점수에 포함 안됨
    - submission 파일 복사 후 날짜 분리, 사용할 feature append
    - 예측
    - submission 파일에 예측값 append
    - 제출

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


In [2]:
# csv load
air_reserve_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/air_reserve.csv.zip')
hpg_reserve_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/hpg_reserve.csv.zip')
air_store_info_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/air_store_info.csv.zip')
hpg_store_info_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/hpg_store_info.csv.zip')
store_id_relation_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/store_id_relation.csv.zip')
date_info_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/date_info.csv.zip')
air_visit_data_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/air_visit_data.csv.zip')
sample_submission_df = pd.read_csv(f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/sample_submission.csv.zip')

In [3]:
air_reserve_df.head()
# print(len(air_reserve_df['air_store_id'].unique()))
# print(len(air_reserve_df['air_store_id']))

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [4]:
air_store_info_df.head()
# print(len(air_store_info_df['air_store_id'].unique()))

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [5]:
hpg_reserve_df.head()
# print(len(hpg_reserve_df['hpg_store_id'].unique()))
# print(len(hpg_reserve_df))

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [6]:
hpg_store_info_df.head()
# print(len(hpg_store_info_df['hpg_store_id'].unique()))

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [7]:
store_id_relation_df.head()
# print(len(store_id_relation_df))

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [8]:
# store_id_relation.csv안에 id들이 1대1 맵핑인지 확인
check_air_id = store_id_relation_df.duplicated(['air_store_id'])
check_hpg_id = store_id_relation_df.duplicated(['hpg_store_id'])
for i, e in enumerate(check_air_id):
    if (e == True):
        print("air_id: " + i)
for i, e in enumerate(check_hpg_id):
    if (e == True):
        print("hpg_id: " + i)

print("Done") # 1대1 맵핑 확인

Done


In [9]:
date_info_df.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [10]:
air_visit_data_df.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [11]:
sample_submission_df.head()
# df = sample_submission_df.copy()
# df['new_id'] = df['id'].str.slice(start=0, stop=20)
# df.head()
# print(len(df['new_id'].unique()))

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## preprocess

### 하나의 csv로 만들기

In [12]:
# air 예약정보와 가게정보 합치기
air_mixed_df = pd.merge(air_reserve_df, air_store_info_df)
# air_mixed_df.head()
# len(air_mixed_df)

In [13]:
hpg_mixed_df = pd.merge(hpg_reserve_df, hpg_store_info_df)
hpg_mixed_df.head()
# print(len(hpg_reserve_df))
# print(len(hpg_mixed_df))

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
1,hpg_dac72789163a3f47,2016-01-02 12:00:00,2016-01-01 20:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
2,hpg_dac72789163a3f47,2016-01-03 19:00:00,2016-01-02 15:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
3,hpg_dac72789163a3f47,2016-01-06 12:00:00,2016-01-06 08:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
4,hpg_dac72789163a3f47,2016-01-10 17:00:00,2016-01-04 22:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698


In [14]:
# column name 공통되게 변경
air_mixed_new_columns_df = air_mixed_df.rename(columns = {"air_store_id":"id", "air_genre_name":"genre", "air_area_name": "area_name"}, inplace = False)
hpg_mixed_new_columns_df = hpg_mixed_df.rename(columns = {"hpg_store_id":"id", "hpg_genre_name":"genre", "hpg_area_name": "area_name"}, inplace = False)
mixed_df = pd.concat([air_mixed_new_columns_df, hpg_mixed_new_columns_df])
mixed_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619


In [34]:
# visit_datetime을 년월일로 변경
mixed_cp_df = mixed_df.copy()
mixed_cp_df['visit_datetime'] =  pd.to_datetime(mixed_cp_df['visit_datetime'])
mixed_cp_df['visit_date'] = [d.date() for d in mixed_cp_df['visit_datetime']]
# mixed_cp_df= mixed_cp_df.drop(['id', 'visit_datetime', 'reserve_datetime'], axis='columns')
mixed_cp_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-08
...,...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24


In [40]:
pd.DataFrame(mixed_cp_df.groupby(by=['visit_date','latitude','longitude','genre','area_name', 'id'], as_index=False).sum())

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors
0,2016-01-01,33.556881,130.415134,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6
1,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6
2,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2
3,2016-01-01,33.579734,130.401762,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2
4,2016-01-01,33.586969,130.392801,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2
...,...,...,...,...,...,...,...
590934,2017-05-31,35.682504,139.773619,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8
590935,2017-05-31,35.691384,139.701256,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4
590936,2017-05-31,35.691384,139.701256,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1
590937,2017-05-31,35.695780,139.768453,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40


In [36]:
# 같은 날, 같은 식당의 reserve_visitors 합치기
merged_df = mixed_cp_df.groupby(by=['visit_date','latitude','longitude','genre','area_name', 'id'], as_index=False).sum()
merged_df.head(40)

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors
0,2016-01-01,33.556881,130.415134,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6
1,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6
2,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2
3,2016-01-01,33.579734,130.401762,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2
4,2016-01-01,33.586969,130.392801,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2
5,2016-01-01,33.586969,130.392801,Japanese style,Fukuoka-ken Fukuoka-shi Daimyō,hpg_c265a318b37114f0,8
6,2016-01-01,33.589806,130.400170,Japanese style,Fukuoka-ken Fukuoka-shi None,hpg_1cc9523ff72fc7cc,8
7,2016-01-01,33.589806,130.400170,Japanese style,Fukuoka-ken Fukuoka-shi None,hpg_28bdf7a336ec6a7b,17
8,2016-01-01,33.593811,130.407306,Japanese style,None None None,hpg_37cd2a03b79e6ba0,19
9,2016-01-01,33.593811,130.407306,Seafood,None None None,hpg_c1d7804e110ab296,4


In [17]:
# 요일과 holiday flag 추가하기
merged_with_day_df = pd.merge(merged_df, date_info_df, left_on='visit_date', right_on='calendar_date')
merged_with_day_df

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors,calendar_date,day_of_week,holiday_flg
0,2016-01-01,33.556881,130.415134,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6,2016-01-01,Friday,1
1,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6,2016-01-01,Friday,1
2,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2,2016-01-01,Friday,1
3,2016-01-01,33.579734,130.401762,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2,2016-01-01,Friday,1
4,2016-01-01,33.586969,130.392801,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2,2016-01-01,Friday,1
...,...,...,...,...,...,...,...,...,...,...
590934,2017-05-31,35.682504,139.773619,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8,2017-05-31,Wednesday,0
590935,2017-05-31,35.691384,139.701256,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4,2017-05-31,Wednesday,0
590936,2017-05-31,35.691384,139.701256,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1,2017-05-31,Wednesday,0
590937,2017-05-31,35.695780,139.768453,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40,2017-05-31,Wednesday,0


In [18]:
# genre 값 확인
merged_with_day_df['genre'].unique()

array(['Seafood', 'Japanese style', 'International cuisine',
       'Grilled meat', 'Italian', 'Dining bar', 'French',
       'Spain Bar/Italian Bar', 'Okonomiyaki/Monja/Teppanyaki',
       'Japanese cuisine/Kaiseki', 'Japanese food', 'Creation',
       'Japanese food in general', 'Shabu-shabu/Sukiyaki', 'Izakaya',
       'Chinese general', 'Creative Japanese food', 'Sushi',
       'Steak/Hamburger/Curry', 'Bistro', 'Udon/Soba', 'Italian/French',
       'Korean cuisine', 'Sweets', 'Karaoke', 'Bar/Cocktail',
       'Cafe/Sweets', 'Cafe', 'Thai/Vietnamese food', 'Pasta/Pizza',
       'Western food', 'Yakiniku/Korean food', 'Party', 'Amusement bar',
       'Cantonese food', 'Sichuan food', 'Shanghai food',
       'Spain/Mediterranean cuisine', 'Dim Sum/Dumplings',
       'Taiwanese/Hong Kong cuisine', 'Other', 'Creative cuisine',
       'Karaoke/Party', 'Asian'], dtype=object)

In [19]:
# area_name none 확인
merged_with_day_df['area_name'].unique()

array(['Fukuoka-ken Fukuoka-shi None',
       'Fukuoka-ken Fukuoka-shi Minamishō',
       'Fukuoka-ken Fukuoka-shi Ōmiya', 'Fukuoka-ken Fukuoka-shi Daimyō',
       'None None None', 'Fukuoka-ken Kitakyūshū-shi Sunatsu',
       'Hiroshima-ken Fukuyama-shi Sannomaruchō',
       'Ōsaka-fu Higashiōsaka-shi Wakaenishishinmachi',
       'Ōsaka-fu Ōsaka-shi Shinsaibashisuji',
       'Ōsaka-fu Ōsaka-shi Shinmachi', 'Hyōgo-ken Kōbe-shi None',
       'Ōsaka-fu Ōsaka-shi None', 'Ōsaka-fu Ōsaka-shi Nakanochō',
       'Osaka Prefecture Osaka None', 'Ōsaka-fu Suita-shi Enokichō',
       'Hyōgo-ken Kakogawa-shi Noguchichō Yoshino',
       'Ōsaka-fu Takatsuki-shi Kamitanabechō',
       'Shizuoka-ken Shizuoka-shi Hinodechō',
       'Shizuoka-ken Shizuoka-shi Irie',
       'Shizuoka-ken Numazu-shi Yanagisawa',
       'Tōkyō-to Machida-shi Nakamachi',
       'Tōkyō-to Shinagawa-ku Yutakachō', 'Tōkyō-to Tama-shi Tsurumaki',
       'Tōkyō-to Minato-ku Takanawa', 'Tōkyō-to Shibuya-ku Ebisunishi',
       'Tō

In [20]:
# air_visit_data 이용해서 워크인, 노쇼 등 확인
check_no_show_df = pd.merge(air_visit_data_df, merged_with_day_df, left_on=['air_store_id', 'visit_date'], right_on=['id','visit_date'])
check_no_show2_df = check_no_show_df.drop(['id', 'latitude', 'longitude', 'genre', 'area_name', 'calendar_date', 'day_of_week', 'holiday_flg'], axis='columns')
check_no_show2_df

,air_store_id,visit_date,visitors,reserve_visitors
0,air_35512c42db0868da,2016-02-27,6,12
1,air_ee3a01f0c71a769f,2016-01-04,61,2
2,air_ee3a01f0c71a769f,2016-01-08,21,3
3,air_ee3a01f0c71a769f,2016-01-09,57,25
4,air_ee3a01f0c71a769f,2016-01-10,32,5
...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,2
28060,air_754ae581ad80cc9f,2017-04-15,39,20
28061,air_754ae581ad80cc9f,2017-04-19,6,6
28062,air_754ae581ad80cc9f,2017-04-22,47,28


In [21]:
# 노쇼와 워크인의 비중이 큰듯하여 예약에 관한 내용을 신경쓰지 않기로 함
temp_df = check_no_show_df.drop(['id', 'calendar_date'], axis='columns')
temp_df

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.566260,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,2,Friday,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,20,Saturday,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,6,Wednesday,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,28,Saturday,0


In [22]:
# area_name none 확인
# temp_df['area_name'].unique()
temp_df['area_name'].value_counts(ascending=True)

Tōkyō-to Taitō-ku Asakusa                         1
Hyōgo-ken Kōbe-shi Motomachidōri                  1
Hyōgo-ken Amagasaki-shi Higashinanamatsuchō       1
Tōkyō-to Setagaya-ku Okusawa                      1
Tōkyō-to Musashino-shi Midorichō                  1
                                               ... 
Shizuoka-ken Hamamatsu-shi Motoshirochō        1349
Hokkaidō Sapporo-shi Minami 3 Jōnishi          1421
Hiroshima-ken Hiroshima-shi Kokutaijimachi     1710
Fukuoka-ken Fukuoka-shi Daimyō                 1801
Tōkyō-to Shibuya-ku Shibuya                    2158
Name: area_name, Length: 71, dtype: int64

In [23]:
# genre 값 확인
# temp_df['genre'].unique()
temp_df['genre'].value_counts(ascending=True)

Asian                              1
International cuisine              3
Karaoke/Party                     43
Other                            248
Creative cuisine                 331
Bar/Cocktail                     426
Okonomiyaki/Monja/Teppanyaki     932
Western food                     950
Cafe/Sweets                     1290
Yakiniku/Korean food            1508
Japanese food                   3076
Dining bar                      3406
Italian/French                  6071
Izakaya                         9779
Name: genre, dtype: int64

In [47]:
# genre 통합
final_df = temp_df.copy()
final_df['genre'] = final_df['genre'].replace('Asian', 'Other')
final_df['genre'] = final_df['genre'].replace('International cuisine', 'Other')
# final_df['genre'].value_counts(ascending=True)
final_df

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.566260,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,2,Friday,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,20,Saturday,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,6,Wednesday,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,28,Saturday,0


In [48]:
final_df['visit_date'] =  pd.to_datetime(final_df['visit_date'])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28064 entries, 0 to 28063
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   air_store_id      28064 non-null  object        
 1   visit_date        28064 non-null  datetime64[ns]
 2   visitors          28064 non-null  int64         
 3   latitude          28064 non-null  float64       
 4   longitude         28064 non-null  float64       
 5   genre             28064 non-null  object        
 6   area_name         28064 non-null  object        
 7   reserve_visitors  28064 non-null  int64         
 8   day_of_week       28064 non-null  object        
 9   holiday_flg       28064 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 2.4+ MB


In [50]:
# Split converted date-time columns to year, month, date, day of week and time separate coluns of dataset
# https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96
# https://www.kaggle.com/irinaabdullaeva/welcome-recruit-restaurant-visitor-forecasting
final_df['visit_year'] = pd.Series(final_df.visit_date.dt.year)
final_df['visit_month']  = pd.Series(final_df.visit_date.dt.month)
final_df['visit_day'] = pd.Series(final_df.visit_date.dt.day)


In [51]:
final_df.head()

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg,visit_year,visit_month,visit_day
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.56626,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0,2016,2,27
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0,2016,1,4
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0,2016,1,8
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0,2016,1,9
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0,2016,1,10


In [52]:
final_df.genre.nunique()

12

In [86]:
final_df.head()

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg,visit_year,visit_month,visit_day
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.56626,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0,2016,2,27
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0,2016,1,4
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0,2016,1,8
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0,2016,1,9
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0,2016,1,10


,id,visitors_x,visit_date,visitors_y,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg,visit_year,visit_month,visit_day


## one hot encoding 


In [164]:
# https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/

one_hot_encoded_genre = pd.get_dummies(final_df, columns = ['genre','area_name','day_of_week'])
one_hot_encoded_df = pd.concat([final_df, one_hot_encoded_genre], axis=1)
oh_final_df= one_hot_encoded_df.drop(['genre','area_name','day_of_week'], axis=1) 
oh_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28064 entries, 0 to 28063
Columns: 110 entries, air_store_id to day_of_week_Wednesday
dtypes: float64(4), int64(12), object(4), uint8(90)
memory usage: 6.9+ MB


In [165]:
oh_final_df.head()

,air_store_id,visit_date,visitors,latitude,longitude,reserve_visitors,holiday_flg,visit_year,visit_month,visit_day,...,area_name_Ōsaka-fu Ōsaka-shi Shinmachi,area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,area_name_Ōsaka-fu Ōsaka-shi Ōhiraki,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.56626,12,0,2016,2,27,...,0,0,0,0,0,1,0,0,0,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.72594,2,0,2016,1,4,...,0,0,0,0,1,0,0,0,0,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.72594,3,0,2016,1,8,...,0,0,0,1,0,0,0,0,0,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.72594,25,0,2016,1,9,...,0,0,0,0,0,1,0,0,0,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.72594,5,0,2016,1,10,...,0,0,0,0,0,0,1,0,0,0


In [103]:
category_features = oh_final_df.columns
category_features.

Index(['air_store_id', 'visit_date', 'visitors', 'latitude', 'longitude',
       'reserve_visitors', 'holiday_flg', 'visit_year', 'visit_month',
       'visit_day',
       ...
       'area_name_Ōsaka-fu Ōsaka-shi Shinmachi',
       'area_name_Ōsaka-fu Ōsaka-shi Ōgimachi',
       'area_name_Ōsaka-fu Ōsaka-shi Ōhiraki', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday'],
      dtype='object', length=110)

In [118]:
# sample_submission 데이터 전처리
# sample_submission_df.head()
temp_df = sample_submission_df.copy()
temp_df['store_id'] = temp_df['id'].str.slice(start=0, stop=20)
temp_df['date'] = temp_df['id'].str.slice(start=21, stop=31)
temp_df = temp_df.drop(['id', 'visitors'], axis='columns')

temp_df



,store_id,date
0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9,2017-04-27
...,...,...
32014,air_fff68b929994bfbd,2017-05-27
32015,air_fff68b929994bfbd,2017-05-28
32016,air_fff68b929994bfbd,2017-05-29
32017,air_fff68b929994bfbd,2017-05-30


In [148]:
date_info_df2 = date_info_df
date_info_df2["calendar_date"] = pd.to_datetime(date_info_df2['calendar_date'])

In [153]:
temp_df2 = pd.merge(temp_df, air_store_info_df, left_on="store_id", right_on="air_store_id")
temp_df2 = temp_df2.rename(columns = {"store_id":"id", "air_genre_name":"genre", "air_area_name": "area_name"}, inplace = False)
temp_df2['date'] =  pd.to_datetime(temp_df2['date'])
temp_df2['visit_date'] = pd.to_datetime([d.date() for d in temp_df2['date']])
temp_df2 = temp_df2.drop(["date","air_store_id"],  axis='columns')


In [154]:
temp_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32019 entries, 0 to 32018
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          32019 non-null  object        
 1   genre       32019 non-null  object        
 2   area_name   32019 non-null  object        
 3   latitude    32019 non-null  float64       
 4   longitude   32019 non-null  float64       
 5   visit_date  32019 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 1.7+ MB


In [155]:
temp_df2 = pd.merge(temp_df2, date_info_df2, left_on='visit_date', right_on='calendar_date')


In [156]:
temp_df2.head()

,id,genre,area_name,latitude,longitude,visit_date,calendar_date,day_of_week,holiday_flg
0,air_00a91d42b08b08d9,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017-04-23,2017-04-23,Sunday,0
1,air_0164b9927d20bcc3,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2017-04-23,2017-04-23,Sunday,0
2,air_0241aa3964b7f861,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2017-04-23,2017-04-23,Sunday,0
3,air_0328696196e46f18,Dining bar,Ōsaka-fu Ōsaka-shi Nakanochō,34.701279,135.528090,2017-04-23,2017-04-23,Sunday,0
4,air_034a3d5b40d5b1b1,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2017-04-23,2017-04-23,Sunday,0


In [157]:
temp_df2 = temp_df2.copy()
temp_df2['genre'] = temp_df2['genre'].replace('Asian', 'Other')
temp_df2['genre'] = temp_df2['genre'].replace('International cuisine', 'Other')

In [158]:
temp_df2['visit_year'] = pd.Series(temp_df2.visit_date.dt.year)
temp_df2['visit_month']  = pd.Series(temp_df2.visit_date.dt.month)
temp_df2['visit_day'] = pd.Series(temp_df2.visit_date.dt.day)

In [166]:
temp_df2['area_name'].value_counts(ascending=True)

Fukuoka-ken Fukuoka-shi Tenjin            39
Tōkyō-to Setagaya-ku Kitazawa             78
Tōkyō-to Fuchū-shi Miyanishichō           78
Tōkyō-to Toshima-ku Mejiro                78
Tōkyō-to Shinagawa-ku Higashigotanda      78
                                        ... 
Tōkyō-to Setagaya-ku Setagaya           1170
Tōkyō-to Shinjuku-ku Kabukichō          1521
Tōkyō-to Minato-ku Shibakōen            1989
Tōkyō-to Shibuya-ku Shibuya             2262
Fukuoka-ken Fukuoka-shi Daimyō          2457
Name: area_name, Length: 103, dtype: int64

In [ ]:
temp_df['genre'].value_counts(ascending=True)

In [159]:
one_hot_encoded_genre = pd.get_dummies(temp_df2, columns = ['genre','area_name','day_of_week'])
one_hot_encoded_df = pd.concat([temp_df2, one_hot_encoded_genre], axis=1)
oh_final_sub = one_hot_encoded_df.drop(['genre','area_name','day_of_week'], axis=1) 

In [160]:
oh_final_sub.head()

,id,latitude,longitude,visit_date,calendar_date,holiday_flg,visit_year,visit_month,visit_day,id,...,area_name_Ōsaka-fu Ōsaka-shi Shinmachi,area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,area_name_Ōsaka-fu Ōsaka-shi Ōhiraki,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,air_00a91d42b08b08d9,35.694003,139.753595,2017-04-23,2017-04-23,0,2017,4,23,air_00a91d42b08b08d9,...,0,0,0,0,0,0,1,0,0,0
1,air_0164b9927d20bcc3,35.658068,139.751599,2017-04-23,2017-04-23,0,2017,4,23,air_0164b9927d20bcc3,...,0,0,0,0,0,0,1,0,0,0
2,air_0241aa3964b7f861,35.712607,139.779996,2017-04-23,2017-04-23,0,2017,4,23,air_0241aa3964b7f861,...,0,0,0,0,0,0,1,0,0,0
3,air_0328696196e46f18,34.701279,135.528090,2017-04-23,2017-04-23,0,2017,4,23,air_0328696196e46f18,...,0,0,0,0,0,0,1,0,0,0
4,air_034a3d5b40d5b1b1,34.692337,135.472229,2017-04-23,2017-04-23,0,2017,4,23,air_034a3d5b40d5b1b1,...,0,0,1,0,0,0,1,0,0,0


### Feature 찾기

In [83]:
sub_id = one_hot_encoded_df['air_store_id']
one_hot_encoded_df= one_hot_encoded_df.drop(['air_store_id','visit_date'], axis=1) 
one_hot_encoded_df.head()

,visitors,latitude,longitude,reserve_visitors,holiday_flg,visit_year,visit_month,visit_day,visitors,latitude,...,area_name_Ōsaka-fu Ōsaka-shi Shinmachi,area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,area_name_Ōsaka-fu Ōsaka-shi Ōhiraki,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,6,35.717784,139.56626,12,0,2016,2,27,6,35.717784,...,0,0,0,0,0,1,0,0,0,0
1,61,34.710895,137.72594,2,0,2016,1,4,61,34.710895,...,0,0,0,0,1,0,0,0,0,0
2,21,34.710895,137.72594,3,0,2016,1,8,21,34.710895,...,0,0,0,1,0,0,0,0,0,0
3,57,34.710895,137.72594,25,0,2016,1,9,57,34.710895,...,0,0,0,0,0,1,0,0,0,0
4,32,34.710895,137.72594,5,0,2016,1,10,32,34.710895,...,0,0,0,0,0,0,1,0,0,0


# train test split


In [68]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error   

train = one_hot_encoded_df.drop(['visitors'], axis=1).values 
y = one_hot_encoded_df['visitors'].values
random_state=2020  
X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=random_state, test_size=0.2)
print('Training shapes:' ,X_train.shape, y_train.shape)
print('Training shapes:' ,X_test.shape, y_test.shape)


Training shapes: (22451, 104) (22451, 2)
Training shapes: (5613, 104) (5613, 2)


In [72]:
X_train

array([[ 43.7706355, 142.364819 ,   6.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 34.3862445, 132.4550176,  23.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 34.3862445, 132.4550176,  33.       , ...,   0.       ,
          0.       ,   1.       ],
       ...,
       [ 34.7108955, 137.7259397,   4.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 38.2690761, 140.8704029,  23.       , ...,   0.       ,
          1.       ,   0.       ],
       [ 34.6951242, 135.1978525,  13.       , ...,   1.       ,
          0.       ,   0.       ]])

## Train

In [79]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

In [81]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth = 5, n_estimators=100, random_state = 42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
# 예측 결과의 rmse값 저장
df= rmse(y_test, y_pred)
# data frame에 저장
# score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)

df

8.351868615637914e+101

## Test

In [66]:
data_dir = f'{os.getenv("HOME")}/aiffel/kaggle_challenge1/'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

Training shapes: (28064, 104) (28064, 2)


In [ ]:
result = pd.DataFrame({
    'id' : sub_id, 
    'price' : y_pred
})

result.head()

In [ ]:
my_submission_path = join(data_dir, 'submission.csv')
result.to_csv(my_submission_path, index=False)

print(my_submission_path)